# MVP author search with OpenAlex data source

The main goal is to return at author (and institution level) from a given search query.

### Groupby and aggregate

Borrowing/Stealing? the design language from pandas, let's think about `groupby` and `aggregate`.

- groupby: defines the return object level (`author` for now)
- aggregate: what formula we will use to `reduce` multiple records into a single metric. For example:
    - basic count of hits
    - custom `reranker` borrowing from retrival augmented generation (RAG) field, we weights the things that people like to see to obtain a score. More specifically, we can obtain the `cited_by_count` in each relevant paper and sum all within an author.
    - `reranker` can also sources form multiple underlying metrics like what we do in [faculty search](https://github.com/UW-Madison-DSI/faculty-search/blob/eff2ecfedcf5e817e70e3f3541b91d4cceeabb27/api/core.py#L387)

TODOs:

1. Make a MVP groupby aggregate interface for search with base hit counts
1. Implement faculty search `reranker` into aggregate


Mock Interface 

```python
search_results = search("effect of fungicide on corn")
search_results.groupby("author").aggregate("count")  # Hit Counts, return authors
search_results.groupby("institution").aggregate("count")  # Hit Counts, return institute
search_results.groupby("author").aggregate("reranker_v0")  # Fully custom reranker, return author
```

In [ ]:
from openalex_search.search import search  # TODO: fix this messy deep import

# This is a simple example of the core of groupby("author").aggregate("count")
search("higgs boson", top_k=3)


[Author(orcid='https://orcid.org/0000-0002-5769-7094', display_name_alternatives=None, cited_by_count=0, updated_date=datetime.datetime(2024, 11, 17, 20, 7, 29, 674881), works_count=0, display_name='K. Cranmer', id='https://openalex.org/A5108167175', last_known_institution=None, works_api_url=None),
 Author(orcid='https://orcid.org/0000-0002-4340-4983', display_name_alternatives=None, cited_by_count=0, updated_date=datetime.datetime(2024, 11, 17, 20, 7, 29, 669944), works_count=0, display_name='Gurtej Kanwar', id='https://openalex.org/A5033100318', last_known_institution=None, works_api_url=None),
 Author(orcid='https://orcid.org/0000-0002-0916-7603', display_name_alternatives=None, cited_by_count=0, updated_date=datetime.datetime(2024, 11, 17, 20, 7, 29, 670844), works_count=0, display_name='Phiala E. Shanahan', id='https://openalex.org/A5080660291', last_known_institution=None, works_api_url=None)]

### Snippets

### Reset DB

In [ ]:
# from sqlmodel import SQLModel
# from pathlib import Path
# from openalex_search.db import init, ENGINE
# from openalex_search.ingest import ingest

# SQLModel.metadata.drop_all(ENGINE)
# init()
# ingest(Path("local_data/test_authors.parquet"))
# ingest(Path("local_data/test_articles.parquet"))